# 数据预处理
:label:`sec_pandas`

到目前为止，我们一直在处理现成张量形式的合成数据。然而，要在实际应用中使用深度学习，我们必须提取以任意格式存储的杂乱数据，并对其进行预处理以满足我们的需求。幸运的是，*pandas* [库](https://pandas.pydata.org/) 可以完成大部分繁重的工作。虽然本节不能替代一个完整的 *pandas* [教程](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)，但它将为您提供一些最常用例程的速成课程。

## 读取数据集

逗号分隔值（CSV）文件在存储表格（类似电子表格）数据时非常普遍。在这些文件中，每一行对应一条记录，由多个（以逗号分隔）字段组成，例如，“Albert Einstein,March 14 1879,Ulm,Federal polytechnic school,field of gravitational physics”。为了演示如何使用 `pandas` 加载 CSV 文件，我们在下面创建了一个名为 `../data/house_tiny.csv` 的 CSV 文件。这个文件代表了一个房屋数据集，其中每一行对应一个不同的房屋，列分别对应房间数量（`NumRooms`）、屋顶类型（`RoofType`）和价格（`Price`）。

In [16]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')

现在让我们导入`pandas`并用`read_csv`加载数据集。

In [17]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000


## 数据准备

在监督学习中，我们训练模型
以预测指定的*目标*值，
给定一些*输入*值。
处理数据集的第一步
是将对应于输入与目标值的列分开。
我们可以按名称或通过基于整数位置的索引（`iloc`）选择列。

你可能已经注意到`pandas`将所有值为`NA`的CSV条目
替换成了一个特殊的`NaN`（*非数字*）值。
当条目为空时，这种情况也可能发生，
例如，“3,,,270000”。
这些被称为*缺失值*
它们是数据科学中的“床虫”，
一种你整个职业生涯都会遇到的持续性威胁。
根据上下文，
缺失值可以通过*插补*或*删除*来处理。
插补用估计值替换缺失值，
而删除则简单地丢弃包含缺失值的那些行或列。

这里有一些常见的插补启发式方法。
[**对于分类输入字段，
我们可以将`NaN`视为一个类别。**]
由于`RoofType`列取值`Slate`和`NaN`，
`pandas`可以将该列转换成两个列`RoofType_Slate`和`RoofType_nan`。
屋顶类型为`Slate`的一行将设置`RoofType_Slate`和`RoofType_nan`的值分别为1和0。
对于缺少`RoofType`值的一行则相反。

In [18]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       NaN           False          True
1       2.0           False          True
2       4.0            True         False
3       NaN           False          True


对于缺失的数值，一个常见的启发式方法是[**将`NaN`条目替换为相应列的平均值**]。

In [19]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0           False          True
1       2.0           False          True
2       4.0            True         False
3       3.0           False          True


## 转换为张量格式

现在，由于`inputs`和`targets`中的所有条目都是数值型的，我们可以将它们加载到张量中（回想 :numref:`sec_ndarray`）。

In [20]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))
X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 讨论

你现在知道如何对数据列进行分区，填补缺失变量，以及将 `pandas` 数据加载到张量中。在 :numref:`sec_kaggle_house` 中，你将学习更多数据处理技能。虽然本速成课程保持了简单，但数据处理可能会变得复杂。例如，我们的数据集可能不是以单个 CSV 文件的形式出现，而是分布在从关系数据库中提取的多个文件中。例如，在电子商务应用中，客户地址可能在一个表中，而购买数据在另一个表中。此外，从业者面对的数据类型除了分类和数值之外还有许多其他类型，例如文本字符串、图像、音频数据和点云。通常需要高级工具和高效算法来防止数据处理成为机器学习管道中的最大瓶颈。当我们接触到计算机视觉和自然语言处理时，这些问题将会出现。最后，我们必须关注数据质量。现实世界的数据集经常受到异常值、传感器故障测量和记录错误的影响，在将数据输入任何模型之前必须解决这些问题。数据可视化工具如 [seaborn](https://seaborn.pydata.org/)、[Bokeh](https://docs.bokeh.org/) 或 [matplotlib](https://matplotlib.org/) 可以帮助你手动检查数据，并对可能需要解决的问题类型产生直觉。

## 练习

1. 尝试加载数据集，例如来自 [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) 的 Abalone 数据集，并检查它们的属性。其中有多少比例有缺失值？多少比例的变量是数值型、类别型或文本型？
1. 尝试通过名称而不是列号索引和选择数据列。pandas 文档中的 [indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) 有关于如何执行此操作的进一步详情。
1. 你认为用这种方式可以加载多大的数据集？可能存在哪些限制？提示：考虑读取数据的时间、表示、处理和内存占用。在你的笔记本电脑上试试看。如果在服务器上尝试会怎样？
1. 如果数据包含非常大量的类别，你会如何处理？如果类别标签都是唯一的呢？你应该包括后者吗？
1. 你能想到 pandas 的哪些替代方案？[从文件加载 NumPy 张量](https://numpy.org/doc/stable/reference/generated/numpy.load.html) 怎么样？查看 [Pillow](https://python-pillow.org/)，即 Python Imaging Library。

[讨论](https://discuss.d2l.ai/t/29)